In [ ]:
import json
import random
from transformers import AutoTokenizer
import numpy as np
from allennlp.common.util import import_module_and_submodules as import_submodules
from allennlp.models.archival import load_archive
from allennlp.predictors import Predictor
from scipy.spatial import distance
from scipy.special import softmax
import ast

import sys
import os
sys.path.append(os.path.abspath('..'))

import_submodules("allennlp_lib")

DATASET="mnli"
MODEL_NAME="roberta-large"
model_path=f"../experiments/models/{DATASET}/{MODEL_NAME}"

archive = load_archive(model_path + '/model.tar.gz')
print(archive.config)
archive.config['dataset_reader']['type'] = 'mnli'
archive.config['model']['output_hidden_states'] = True
model = archive.model
model._output_hidden_states = True
predictor = Predictor.from_archive(archive, 'textual_entailment_fixed')

tok = AutoTokenizer.from_pretrained("roberta-large")

with open(model_path + "/label2index.json", "r") as f:
    label2index = json.load(f)
    index2label = {label2index[k]: k for k in label2index}
label2index


In [ ]:

path = "../data/nli_negation_concept/entailment.jsonl"
path2 = "../data/nli_negation_concept/entailment_with_negation.jsonl"

dneg = []
for line in open(path):
    dneg.append(ast.literal_eval(line.strip()))
    
cneg = []
for line in open(path2):
    cneg.append(ast.literal_eval(line.strip()))
    
avg = 0

assert len(cneg)==len(dneg)
print("Numer of examples: ", len(cneg))

count = 0

for ex,ex2 in zip(dneg,cneg):
    print(ex)
    ex['sentence1'] = ex['Sentence1']
    ex['sentence2'] = ex['Sentence2']
    ex2['sentence1'] = ex2['Sentence1']
    ex2['sentence2'] = ex2['Sentence2']
    
    out = predictor.predict_json(ex)
    out2 = predictor.predict_json(ex2)

    count += 1
    print('Revised sentence:', ex2['Sentence2'])
    print(out['label'], '-->', out2['label'])
    avg += out['probs'][1] - out2['probs'][1]

    print(f"{out['probs'][1] * 100:.2f}% --> {out2['probs'][1] * 100:.2f}%. (e: {out2['probs'][0] * 100:.2f}%, n:{out2['probs'][2] * 100:.2f}%)")
    print()
    
avg = avg / count
print(f"{avg*100:.2f}%")
print(count)


In [ ]:

path = "../data/nli_negation_concept/neutral.jsonl"
path2 = "../data/nli_negation_concept/neutral_with_negation.jsonl"

dneg = []
for line in open(path):
    dneg.append(ast.literal_eval(line.strip()))
    
cneg = []
for line in open(path2):
    cneg.append(ast.literal_eval(line.strip()))

avg = 0

assert len(cneg)==len(dneg)
print("Numer of examples: ", len(cneg))

count = 0

for ex,ex2 in zip(dneg,cneg):
    print(ex)
    ex['sentence1'] = ex['Sentence1']
    ex['sentence2'] = ex['Sentence2']
    ex2['sentence1'] = ex2['Sentence1']
    ex2['sentence2'] = ex2['Sentence2']
    
    out = predictor.predict_json(ex)
    out2 = predictor.predict_json(ex2)

    count += 1
    print('Revised sentence:', ex2['Sentence2'])
    print(out['label'], '-->', out2['label'])
    avg += out['probs'][1] - out2['probs'][1]

    print(f"{out['probs'][1] * 100:.2f}% --> {out2['probs'][1] * 100:.2f}%. (e: {out2['probs'][0] * 100:.2f}%, n:{out2['probs'][2] * 100:.2f}%)")
    print()
    
avg = avg / count
print(f"{avg*100:.2f}%")
print(count)
